# get_elevation_shp 함수

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
from pyproj import Transformer


# ✅ 등고선 shapefile 경로 (shp 파일만 지정하면 나머지 자동 인식됨)
file_path = "/home/dan/dev_ws/EDA/project/data/Topo/서울시 등고선/등고선 5000/N3L_F001.shp"

# ✅ GeoDataFrame으로 불러오기
contours = gpd.read_file(file_path)


def get_elevation_shp(lng, lat):

    
    # WGS84(EPSG:4326)에서 EPSG:5174로 변환기 생성
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:5174", always_xy=True)

    # 좌표 변환 (경도, 위도) -> (x, y)
    x, y = transformer.transform(lng, lat)

    # 변환된 좌표로 포인트 생성
    target_point = Point(x, y)
    # print("변환된 좌표:", x, y)


    # 가장 가까운 등고선 찾기
    distances = contours.geometry.distance(target_point)
    closest_idx = distances.idxmin()
    elevation = contours.iloc[closest_idx]['HEIGHT']

    # print(f"해당 좌표의 고도: {elevation}m")
    # print("등고선 데이터 좌표계:", contours.crs)
    # print("사용 중인 좌표:", target_point)
    
    return elevation


# DB 의 station 정보 불러오기

In [13]:
import mysql.connector

remote = mysql.connector.connect (
    host = "jinhyuk2db.c94oygiiukcg.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "root",
    password = "jinhyuk2dacibul",
    database = "project1"
)

cursor = remote.cursor(buffered=True)

cursor.execute("SELECT * \
                FROM station\
                ORDER BY station_id;")

result = cursor.fetchall()

for row in result:
    print(row)

('ST-10', '마포구', 37.552746, 126.918617, None, '서교동')
('ST-100', '광진구', 37.536667, 127.073593, None, '자양동')
('ST-1000', '양천구', 37.51038, 126.866798, None, '신정동')
('ST-1001', '양천구', 37.5297061, 126.8324456, None, '신월동')
('ST-1002', '양천구', 37.5299, 126.876541, None, '목동')
('ST-1003', '양천구', 37.539551, 126.8283, None, '신월동')
('ST-1004', '양천구', 37.514099, 126.831001, None, '신정동')
('ST-1005', '양천구', 37.51395, 126.856056, None, '신정동')
('ST-1006', '양천구', 37.536377, 126.871513, None, '목동')
('ST-1007', '양천구', 37.52219, 126.8367, None, '신월동')
('ST-1008', '양천구', 37.522312, 126.849045, None, '신정동')
('ST-1009', '양천구', 37.5365481, 126.8289314, None, '신월동')
('ST-101', '광진구', 37.53297, 127.075935, None, '자양동')
('ST-1010', '양천구', 37.536995, 126.827286, None, '신월동')
('ST-1011', '양천구', 37.536369, 126.831711, None, '신월동')
('ST-1012', '양천구', 37.53952, 126.825401, None, '신월동')
('ST-1013', '양천구', 37.550732, 126.864578, None, '목동')
('ST-1014', '양천구', 37.5089, 126.842682, None, '신정동')
('ST-1015', '양천구', 37.5438

In [53]:
import pandas as pd

df_station = pd.DataFrame(result, columns = ['id', 'gu', 'lat', 'lng', 'elev', 'dong'], )
# df_station.drop(['대여날', '날씨날'], axis=1, inplace=True)
df_station

,id,gu,lat,lng,elev,dong
0,ST-10,마포구,37.552746,126.918617,None,서교동
1,ST-100,광진구,37.536667,127.073593,None,자양동
2,ST-1000,양천구,37.510380,126.866798,None,신정동
3,ST-1001,양천구,37.529706,126.832446,None,신월동
4,ST-1002,양천구,37.529900,126.876541,None,목동
...,...,...,...,...,...,...
3294,ST-995,양천구,37.510597,126.857323,None,신정동
3295,ST-996,양천구,37.524334,126.850548,None,신월동
3296,ST-997,양천구,37.534390,126.869598,None,목동
3297,ST-998,양천구,37.531931,126.871591,None,목동


# 각 station 좌표로 고도 뽑기

In [68]:
from tqdm import tqdm

elevations = []

for i in tqdm(range(len(df_station))):
    elev = get_elevation_shp(lng=df_station['lng'][i], lat=df_station['lat'][i])
    elevations.append(elev)

100%|██████████| 3299/3299 [03:50<00:00, 14.30it/s]


기존 df_station 데이터프레임에 고도 정보 삽입

In [82]:
df_station['elev'] = elevations
df_station

,id,gu,lat,lng,elev,dong
0,ST-10,마포구,37.552746,126.918617,15.0,서교동
1,ST-100,광진구,37.536667,127.073593,15.0,자양동
2,ST-1000,양천구,37.510380,126.866798,10.0,신정동
3,ST-1001,양천구,37.529706,126.832446,20.0,신월동
4,ST-1002,양천구,37.529900,126.876541,10.0,목동
...,...,...,...,...,...,...
3294,ST-995,양천구,37.510597,126.857323,25.0,신정동
3295,ST-996,양천구,37.524334,126.850548,30.0,신월동
3296,ST-997,양천구,37.534390,126.869598,10.0,목동
3297,ST-998,양천구,37.531931,126.871591,10.0,목동


# CSV로 df_station 저장 (utf-8)

In [83]:
df_station.to_csv("/home/dan/dev_ws/EDA/project/data/Rental/station_with_elevation.csv", 
                sep=",", index=False, encoding="utf-8")

# DB Table (station_B) 만들기

In [85]:
import mysql.connector

remote = mysql.connector.connect (
    host = "jinhyuk2db.c94oygiiukcg.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "root",
    password = "jinhyuk2dacibul",
    database = "project1"
)

cursor = remote.cursor(buffered=True)

cursor.execute("CREATE TABLE station_B (station_id varchar(10) NOT NULL PRIMARY KEY,\
               station_gu varchar(8) NOT NULL,\
               station_latitude double NOT NULL,\
               station_longitude double NOT NULL,\
               station_elevation double,\
               station_dong varchar(8))")

remote.commit()
remote.close()

# DB에 데이터 집어넣기

In [86]:
import mysql.connector

remote = mysql.connector.connect (
    host = "jinhyuk2db.c94oygiiukcg.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "root",
    password = "jinhyuk2dacibul",
    database = "project1"
)

cursor = remote.cursor(buffered=True)


sql= """INSERT INTO station_B VALUES (%s, %s, %s, %s, %s, %s)"""

for i,row in df_station.iterrows(): 

    id = row['id']
    gu = row['gu']
    lat = row['lat']
    lng = row['lng']
    elev = row['elev']
    dong = row['dong']

    content = (id, gu, lat, lng, elev, dong)

    cursor.execute(sql, content)

    print(content)
    

remote.commit()
remote.close()

('ST-10', '마포구', 37.552746, 126.918617, 15.0, '서교동')
('ST-100', '광진구', 37.536667, 127.073593, 15.0, '자양동')
('ST-1000', '양천구', 37.51038, 126.866798, 10.0, '신정동')
('ST-1001', '양천구', 37.5297061, 126.8324456, 20.0, '신월동')
('ST-1002', '양천구', 37.5299, 126.876541, 10.0, '목동')
('ST-1003', '양천구', 37.539551, 126.8283, 20.0, '신월동')
('ST-1004', '양천구', 37.514099, 126.831001, 40.0, '신정동')
('ST-1005', '양천구', 37.51395, 126.856056, 15.0, '신정동')
('ST-1006', '양천구', 37.536377, 126.871513, 10.0, '목동')
('ST-1007', '양천구', 37.52219, 126.8367, 20.0, '신월동')
('ST-1008', '양천구', 37.522312, 126.849045, 25.0, '신정동')
('ST-1009', '양천구', 37.5365481, 126.8289314, 15.0, '신월동')
('ST-101', '광진구', 37.53297, 127.075935, 15.0, '자양동')
('ST-1010', '양천구', 37.536995, 126.827286, 20.0, '신월동')
('ST-1011', '양천구', 37.536369, 126.831711, 15.0, '신월동')
('ST-1012', '양천구', 37.53952, 126.825401, 20.0, '신월동')
('ST-1013', '양천구', 37.550732, 126.864578, 15.0, '목동')
('ST-1014', '양천구', 37.5089, 126.842682, 20.0, '신정동')
('ST-1015', '양천구', 37.5438